# ensegment

This task uses simple word counts to segment strings into the most likely sequence of words.

#### The score reachs 1.00 for dev.txt, 0.97 for test.txt respectively.

## Data Preparation
To check the results of input "data/input/test.txt", we have manually segmented the string in "data/reference/test.out" as a reference.

## Dev Input

This part sets dev.txt as input and prints out the segmentation result. First, import segmentation and support functions:

In [3]:
from ensegment import * 

Then define token number, instantiate class Pdist and Segment, print out the segmented results:

In [4]:
N = 1024908267229
Pw = Pdist(data=datafile("../data/count_1w.txt", sep='\t'),N=N, missingfn = reduce_probability_for_long_word)
segmenter = Segment(Pw)
with open("../data/input/dev.txt") as f:
    for line in f:
        print(" ".join(segmenter.segment(line.strip())))

choose spain
this is a test
who represents
experts exchange
speed of art
un climate change body
we are the people
mention your faves
now playing
the walking dead
follow me
we are the people
mention your faves
check domain
big rock
name cheap
apple domains
honesty hour
being human
follow back
social media
30 seconds to earth
current rate sought to go down
this is insane
what is my name
is it time
let us go
me too
now thatcher is dead
advice for young journalists


#### According to check.py, the score is 1.00

## Test Input

This part sets test.txt as input and prints out the segmentation result.

In [5]:
N = 1024908267229
Pw = Pdist(data=datafile("../data/count_1w.txt", sep='\t'),N=N, missingfn = reduce_probability_for_long_word)
segmenter = Segment(Pw)
with open("../data/input/test.txt") as f:
    for line in f:
        print(" ".join(segmenter.segment(line.strip())))

how to breakup in 5 words
what makes god smile
10 people who mean alot to me
worst day in 4 words
love story in 5 words
top 3 favourite comics
10 breakup lines
things that make you smile
best female athlete
worst boss in 5 words
now is the time for all good
it is a truth universally acknowledged
when in the course of human events it becomes necessary
it was a bright cold day in april and the clocks were striking thirteen
it was the best of times it was the worst of times it was the age of wisdom it was the age of foolishness
as gregor samsa awoke one morning from uneasy dreams he found himself transformed in his bed into a gigantic insect
in a hole in the ground there lived a hobbit not a nasty dirty wet hole filled with the ends of worms and an oozy smell nor yet a dry bare sandy hole with nothing in it to sitdown on or to eat it was a hobbit hole and that means comfort
far out in the uncharted backwaters of the unfashionable end of the western spiral arm of the galaxy lies a small un

#### According to check.py, the score is 0.97

## Analysis



Before reaching the final score as shown above, we have tried the default solution and an improved solution as well. After analyzing the output of the previous solutions, a final solution is then introduced. 

### 1. The Defaut Solution

Below lists the segmented output given by the default solution for dev.txt:

#### According to check.py, the score is 0.82 for dev.txt

As shown above, most strings with 2 words are segmented correctly, but long strings like "unclimatechangebody", "mentionyourfaves", "30secondstoearth" remains unsegmented due to the reason that the product of the P(w) of possible segemented words (e.g. "un", "climate"," "change", "body") is less than the probabilty of a unseen word like "mentionyourfaves", which is 1/N given by the default solution. 

The same issue occurs with input test.txt,  below is the segmentation given by default solution for test.txt:

#### According to check.py, the score is 0.13 for test.txt

To tackle this issue, we have to decrease the probability of unseen words. In addition, with the length of the unseen words growing, like in "top3favouritecomics", some words that do exist in the Corpus will be included in the unseen word, which decreases the chance of finding the right segmentation. So, instead of giving every unseen word the same probabilty, we decrease of the probability of an unseen word according to the length of the word, which leads to the improved solution. 

## 2. The Improved Solution

We first take a look at the code in default.py, which explains how the probability of a unseen word is calculated:

In [6]:
class Pdist(dict):
    "A probability distribution estimated from counts in datafile."
    def __init__(self, data=[], N=None, missingfn=None):
        for key,count in data:
            self[key] = self.get(key, 0) + int(count)
        self.N = float(N or sum(self.values()))
        self.missingfn = missingfn or (lambda k, N: 1./N)
    def __call__(self, key): 
        if key in self: return self[key]/self.N  
        else: return self.missingfn(key, self.N)

As shown above, missingfn is responsible to generate the probability of an unseen word, which assigns a constant 1./N to every unseen word. So we add a new function reduce_probability_for_long_word(word, N) to replace missingfn which decrease the probabilty of an unseen word by (100000 **len(word))

In [7]:
def reduce_probability_for_long_word(word, N):
	return 1./(N * 100000 **len(word))

By applying the reduce_probability_for_long_word(word, N) function with input dev.txt, we get the outputs below:

In [8]:
N = 1024908267229
Pw = Pdist(data=datafile("../data/count_1w.txt", sep='\t'),N=N, missingfn = reduce_probability_for_long_word)
segmenter = Segment(Pw)
with open("../data/input/dev.txt") as f:
    for line in f:
        print(" ".join(segmenter.segment(line.strip())))

choose spain
this is a test
who represents
experts exchange
speed of art
un climate change body
we are the people
mention your faves
now playing
the walking dead
follow me
we are the people
mention your faves
check domain
bigrock
name cheap
apple domains
honesty hour
being human
follow back
social media
30 seconds to earth
current rate sought to go down
this is insane
what is my name
is it time
let us go
me too
now thatcher is dead
advice for young journalists


#### According to check.py, the score is 0.98. 

In the output, only one word is not correctly segmented, i.e. "bigrock" is not segmented into "big rock"

By applying the reduce_probability_for_long_word(word, N) function with input test.txt, we get the outputs below:

In [9]:
N = 1024908267229
Pw = Pdist(data=datafile("../data/count_1w.txt", sep='\t'),N=N, missingfn = reduce_probability_for_long_word)
segmenter = Segment(Pw)
with open("../data/input/test.txt") as f:
    for line in f:
        print(" ".join(segmenter.segment(line.strip())))

howto breakup in 5 words
what makes god smile
10 people who mean alot to me
worst day in 4 words
love story in 5 words
top 3 favourite comics
10 breakup lines
things that make you smile
best female athlete
worst boss in 5 words
now is the time for all good
it is a truth universally acknowledged
when in the course of human events it becomes necessary
it was a bright cold day in april and the clocks were striking thirteen
it was the best of times it was the worst of times it was the age of wisdom it was the age of foolishness
as gregor samsa awoke one morning from uneasy dreams he found himself transformed in his bed into a gigantic insect
in a hole in the ground there lived a hobbit not a nasty dirty wet hole filled with the ends of worms and an oozy smell nor yet a dry bare sandy hole with nothing in it to sitdown on or to eat it was a hobbit hole and that means comfort
far out in the uncharted backwaters of the unfashionable end of the western spiral arm of the galaxy lies a small un 

#### According to check.py, the score is 0.96.

Phrases 1-4 like "break up" and "breakup" both have apperance in the Corpus, so we can not solve this by improving the missingfn. Given a  Corpus, the unigram probability of "breakup" and the product of "break"&"up" is a fixed number. In this particular task, to seperate phrases 1-4, we can decrease the probability of word according to its length so that the P(w) product of short words like "break"&"up" can surpass that of "breakup"  

Phrase 5 "unregarded" has no appearance in the Corpus. The P(w) product of "un"&"regarded" is larger than that of the unseen word, so unregarded is not segmented correctly.

## 3. The Final Solution

Given a word in the Corpus, we change its P(w) from count/N to count/N/(len(w)/3). If the length of a word is less than 3, it's P(w) will be increased; If the length of a word is larger than 3, its P(w) will be decreased, thus seperating short words from phrases.

In [10]:
class Pdist(dict):
    "A probability distribution estimated from counts in datafile."
    def __init__(self, data=[], N=None, missingfn=None):
        for key,count in data:
            self[key] = self.get(key, 0) + int(count)
        self.N = float(N or sum(self.values()))
        self.missingfn = missingfn or (lambda k, N: 1./N)
    def __call__(self, key): 
        if key in self: 
            return self[key]/(self.N *len(key)/3)
        else:
            return self.missingfn(key, self.N)

Applying this idea, we get the final results as shown in the "Dev Input" and "Test Input" section, reaching a score of 1.00 and 0.97 respectively.

For dev.txt, the strings are segmented perfectly. For test.txt, the outputs are listed below:

In [13]:
N = 1024908267229
Pw = Pdist(data=datafile("../data/count_1w.txt", sep='\t'),N=N, missingfn = reduce_probability_for_long_word)
segmenter = Segment(Pw)
with open("../data/input/test.txt") as f:
    for line in f:
        print(" ".join(segmenter.segment(line.strip())))

how to breakup in 5 words
what makes god smile
10 people who mean alot to me
worst day in 4 words
love story in 5 words
top 3 favourite comics
10 breakup lines
things that make you smile
best female athlete
worst boss in 5 words
now is the time for all good
it is a truth universally acknowledged
when in the course of human events it becomes necessary
it was a bright cold day in april and the clocks were striking thirteen
it was the best of times it was the worst of times it was the age of wisdom it was the age of foolishness
as gregor samsa awoke one morning from uneasy dreams he found himself transformed in his bed into a gigantic insect
in a hole in the ground there lived a hobbit not a nasty dirty wet hole filled with the ends of worms and an oozy smell nor yet a dry bare sandy hole with nothing in it to sitdown on or to eat it was a hobbit hole and that means comfort
far out in the uncharted backwaters of the unfashionable end of the western spiral arm of the galaxy lies a small un

"howto" is correctly segmented this time. For dev.txt, "bigrock" is correctly segmented into "big rock". For other phrases, there is still room for improvement.

## Conclusion